### Reference
- Multi-Label Classification Model From Scratch: Step-by-Step Tutorial (https://huggingface.co/blog/Valerii-Knowledgator/multi-label-classification)
- https://github.com/NielsRogge/Transformers-Tutorials/blob/master/BERT/Fine_tuning_BERT_(and_friends)_for_multi_label_text_classification.ipynb
- https://github.com/huggingface/notebooks/blob/main/examples/text_classification.ipynb

# Libraries

In [1]:
%pip install -q -U datasets transformers accelerate sentencepiece # evaluate

In [2]:
import os
import random
import numpy as np
# import pandas as pd
import torch
import transformers
# import evaluate
from pprint import pprint
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, Trainer, TrainingArguments
from sklearn.metrics import f1_score

# Config

In [30]:
seed = 42
lang = 'eng'
# hf_model_id = 'google-bert/bert-base-uncased'
# hf_model_id = 'google-bert/bert-large-uncased'
hf_model_id = 'bhadresh-savani/bert-base-uncased-emotion'
hf_data_id = 'alxxtexxr/SemEval2025-Task11-Dataset'
hf_data_config = 'track_a_eng_70_15_15'
project_name = f'BERT-Base-SE2025T11A-{lang}-v0.5'

In [4]:
def set_seed(seed):
    # Set random seed for NumPy
    np.random.seed(seed)

    # Set random seed for Torch
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if using multi-GPU
    torch.backends.cudnn.deterministic = True  # Ensures deterministic results
    torch.backends.cudnn.benchmark = False  # Avoids non-deterministic algorithms

    # Set random seed for Transformers
    transformers.set_seed(seed)

    # Optionally set random seed for sklearn and Python's own random module
    random.seed(seed)

    # Set random seed for os
    os.environ['PYTHONHASHSEED'] = str(seed)

    print(f"Random seed set to: {seed}")

set_seed(seed)

Random seed set to: 42


# Data

## Load Data

In [5]:
datasets = load_dataset(hf_data_id, hf_data_config)

cols = list(datasets['train'].features)
emotion_cols = [col for col in cols if col not in ['Unnamed: 0', 'text', 'emotion']]

# splits = data_files.keys()
# df = {split: pd.DataFrame(datasets[split]) for split in splits}

# cols = list(df['train'].columns)
print("Data columns:", cols)

# emotion_cols = [col for col in cols if col not in ['Unnamed: 0', 'text', 'emotion']]
# # neutral_emotion = df['train'][df['train'][emotion_cols].sum(axis=1) == 0]['emotion'].iloc[0]
# # emotions = emotion_cols + [neutral_emotion]
print("Emotions columns:", emotion_cols)
# print()

# print("Train DF size:", len(df['train']))
# print("Validation DF size:", len(df['val']))
# print("Testing DF size:", len(df['test']))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:99: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


Data columns: ['text', 'emotion', 'anger', 'fear', 'joy', 'sad', 'surprise']
Emotions columns: ['anger', 'fear', 'joy', 'sad', 'surprise']


In [6]:
class2id = {class_:id for id, class_ in enumerate(emotion_cols)}
id2class = {id:class_ for class_, id in class2id.items()}

print("Class to ID:\n")
pprint(class2id, width=1)
print()
print("ID to Class:\n")
pprint(id2class, width=1)

Class to ID:

{'anger': 0,
 'fear': 1,
 'joy': 2,
 'sad': 3,
 'surprise': 4}

ID to Class:

{0: 'anger',
 1: 'fear',
 2: 'joy',
 3: 'sad',
 4: 'surprise'}


## Preprocess Data

In [7]:
tokenizer = AutoTokenizer.from_pretrained(hf_model_id)

In [8]:
def one_hot_encode_emotion(emotion, emotion_cols):
    emotions = emotion.replace(" ", "").split(",")
    one_hot_emotion = [1.0 if emotion_col in emotions else 0.0 for emotion_col in emotion_cols] # Ensure that the label is float, not int
    return one_hot_emotion

def preprocess_function(data):
   text = data['text']
   emotion = data['emotion']
   labels = one_hot_encode_emotion(emotion, emotion_cols)
   data = tokenizer(text, truncation=True)
   data['labels'] = labels
   return data

tokenized_datasets = {split: datasets[split].map(preprocess_function) for split in ['train', 'val', 'test']}

Map:   0%|          | 0/1937 [00:00<?, ? examples/s]

Map:   0%|          | 0/415 [00:00<?, ? examples/s]

Map:   0%|          | 0/416 [00:00<?, ? examples/s]

In [9]:
# Sanity check
data = tokenized_datasets['train'][0]

print("Text:", data['text'])
print("Emotion(s):", data['emotion'])
print("Labels:", data['labels'], '-->', emotion_cols)

Text: My knuckles growing whiter and whiter, my mind screaming at me that we should be out of here, we should almost be home.
Emotion(s): anger, fear
Labels: [1.0, 1.0, 0.0, 0.0, 0.0] --> ['anger', 'fear', 'joy', 'sad', 'surprise']


In [10]:
data_collator = DataCollatorWithPadding(tokenizer)

# Model

In [22]:
model = AutoModelForSequenceClassification.from_pretrained(
    hf_model_id, 
    # num_labels=len(emotion_cols),
    # id2label=id2class, label2id=class2id,
    problem_type = "multi_label_classification",
)

In [23]:
# for param in model.parameters():
#     param.requires_grad = False
model.classifier = torch.nn.Linear(in_features=768, out_features=len(emotion_cols), bias=True)

# Finetuning

In [24]:
def sigmoid(x):
   return 1/(1 + np.exp(-x))

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    probs = sigmoid(predictions)
    y_pred = (probs > 0.5).astype(int)
    y_true = labels.astype(int)

    # Compute F1 score for each type of averaging method
    f1_micro = f1_score(y_true, y_pred, average='micro', zero_division=0.0)
    f1_macro = f1_score(y_true, y_pred, average='macro', zero_division=0.0)
    # f1_weighted = f1_score(y_true, y_pred, average='weighted', zero_division=0.0)
    # f1_samples = f1_score(y_true, y_pred, average='samples', zero_division=0.0)
    f1_labels = f1_score(y_true, y_pred, average=None, zero_division=0.0)
    f1_labels_dict = {f'f1_label_{emotion_cols[i]}': f1_labels[i] for i in range(len(f1_labels))}

    return {
        'f1_micro': f1_micro,
        'f1_macro': f1_macro,
        **f1_labels_dict,
    }

In [26]:
train_args = TrainingArguments(
    # Training config
    per_device_train_batch_size=2,
    num_train_epochs=2,
    learning_rate=2e-5,
    weight_decay=0.01,

    # Logging config for training
    logging_strategy='steps',
    logging_steps=50,

    # Evaluation config during training
    per_device_eval_batch_size=2,
    eval_strategy='steps',
    eval_steps=50,

    # Model saving config
    output_dir=project_name,
    save_strategy='epoch',
    # load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['val'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

<ipython-input-26-d7f5c93b8dd2>:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [27]:
trainer.train()

Step,Training Loss,Validation Loss,F1 Micro,F1 Macro,F1 Label Anger,F1 Label Fear,F1 Label Joy,F1 Label Sad,F1 Label Surprise
50,0.618700,0.565652,0.466235,0.176883,0.000000,0.734756,0.000000,0.149660,0.000000
100,0.561000,0.528863,0.510259,0.250003,0.000000,0.733945,0.000000,0.454054,0.062016
150,0.527100,0.504681,0.559353,0.304708,0.039216,0.778157,0.000000,0.644628,0.061538
200,0.490200,0.495451,0.619048,0.422537,0.075472,0.799308,0.306452,0.664789,0.266667
250,0.502600,0.455279,0.603464,0.467929,0.107143,0.746835,0.606383,0.669492,0.209790
300,0.457000,0.455153,0.666667,0.516915,0.172414,0.785358,0.306452,0.718310,0.602041
350,0.404000,0.423405,0.674138,0.548277,0.111111,0.796992,0.588235,0.618182,0.626866
400,0.401600,0.408826,0.694650,0.581457,0.140351,0.801527,0.654762,0.674699,0.635945
450,0.437000,0.416980,0.685714,0.579983,0.175439,0.800805,0.680203,0.582524,0.660944
500,0.414700,0.403341,0.708300,0.638705,0.432432,0.809074,0.647727,0.708333,0.595960


TrainOutput(global_step=1938, training_loss=0.36503237713478176, metrics={'train_runtime': 260.9781, 'train_samples_per_second': 14.844, 'train_steps_per_second': 7.426, 'total_flos': 55199308434264.0, 'train_loss': 0.36503237713478176, 'epoch': 2.0})

In [31]:
trainer.push_to_hub()

events.out.tfevents.1730043424.53a70d13144a.21460.0:   0%|          | 0.00/38.9k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Upload 6 LFS files:   0%|          | 0/6 [00:00<?, ?it/s]

events.out.tfevents.1730043586.53a70d13144a.21460.1:   0%|          | 0.00/751 [00:00<?, ?B/s]

events.out.tfevents.1730043914.53a70d13144a.21460.3:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

events.out.tfevents.1730043646.53a70d13144a.21460.2:   0%|          | 0.00/38.9k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.24k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/alxxtexxr/BERT-Base-SE2025T11A-eng-v0.4/commit/543efe1e6aca7cc65e04297b9a78460d034e1be5', commit_message='End of training', commit_description='', oid='543efe1e6aca7cc65e04297b9a78460d034e1be5', pr_url=None, pr_revision=None, pr_num=None)

# Evaluation

In [29]:
trainer.evaluate(eval_dataset=tokenized_datasets['test'])

{'eval_loss': 0.4005555510520935,
 'eval_f1_micro': 0.7371695178849145,
 'eval_f1_macro': 0.6942271204292378,
 'eval_f1_label_anger': 0.5783132530120482,
 'eval_f1_label_fear': 0.8336673346693386,
 'eval_f1_label_joy': 0.6764705882352942,
 'eval_f1_label_sad': 0.6639344262295082,
 'eval_f1_label_surprise': 0.71875,
 'eval_runtime': 2.907,
 'eval_samples_per_second': 143.101,
 'eval_steps_per_second': 71.551,
 'epoch': 2.0}

# Inference

In [32]:
data = datasets['val'][4]
text = data['text']
emotion_true = data['emotion']

inputs = tokenizer(text, return_tensors='pt').to(model.device)

outputs = trainer.model(**inputs)
logits = outputs.logits
probs = sigmoid(logits.squeeze().detach().cpu().numpy()) # apply sigmoid + threshold
labels_pred = (probs > 0.5).astype(int)
emotion_pred = [id2class[idx] for idx, label in enumerate(labels_pred) if label == 1.0] # turn predicted id's into actual label names

print("Text:", text)
print("True emotion(s):", emotion_true)
print("Predicted emotion(s):", ", ".join(emotion_pred))

Text: I live completely alone, and as I pull the popcorn out of the pantry, I hear an ominously distinct "SSSSHHHH" sound.
True emotion(s): fear, surprise
Predicted emotion(s): fear, surprise
